In [25]:
import pandas as pd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn import metrics
import bisect

import warnings


df = pd.read_csv('data/train.csv')

X_train = df.drop(columns=["Survived"], axis=1)
X_test = pd.read_csv('data/test.csv')
y_train = df["Survived"]
# Supprimer les lignes contenant un '?'

# fnlwgt is a weight variable that represents the demographic characteristics of each person surveyed.
df.head()
df.shape
df.describe()
df.isna().sum()
# Ajouter un titre et des Ã©tiquettes
X_test_passenger_id = X_test["PassengerId"].copy()



# Utiliser la colonne PassengerId sauvegardée


numeric_columns = df.dtypes[((df.dtypes=="float64")|(df.dtypes=="int64"))].index.values.tolist()


    
categorical_col = df.dtypes[df.dtypes=="object"].index.values.tolist()


mmc = StandardScaler()

scaled_columns = [x for x in numeric_columns if x != 'Survived']
print(numeric_columns)

X_train[scaled_columns] = mmc.fit_transform(X_train[scaled_columns])
X_test[scaled_columns] = mmc.transform(X_test[scaled_columns])



ohe = OneHotEncoder(drop = None, handle_unknown='ignore', sparse_output=False)

ohe_cols = [col for col in categorical_col if col in X_test.columns]


X_train_encoded = pd.DataFrame(ohe.fit_transform(X_train[ohe_cols]), index=X_train.index)
X_test_encoded = pd.DataFrame(ohe.transform(X_test[ohe_cols]), index=X_test.index)

X_train_encoded.columns = ohe.get_feature_names_out(ohe_cols)
X_test_encoded.columns = ohe.get_feature_names_out(ohe_cols)

X_train.drop(ohe_cols ,axis=1, inplace=True)
X_test.drop(ohe_cols ,axis=1, inplace=True)

X_train = pd.concat([X_train, X_train_encoded], axis=1)
X_test = pd.concat([X_test, X_test_encoded ], axis=1)

# Définir le grille de paramètres pour Random Forest
random_grid = {
    'n_estimators': [int(x) for x in np.linspace(start=50, stop=400, num=8)],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [int(x) for x in np.linspace(10, 110, num=11)] + [None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
    'class_weight': [None, 'balanced'],  
    'max_leaf_nodes': [None] + [int(x) for x in np.linspace(10, 100, num=10)],  
    'min_weight_fraction_leaf': [0.0, 0.1],  
    'criterion': ['gini', 'entropy'] 
}

# Rechercher les meilleurs paramètres
rf = RandomForestClassifier(random_state=0)
rf_search = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=10, cv=3, verbose=1, random_state=0)
rf_search.fit(X_train, y_train)

# Prédictions
y_test_pred = rf_search.predict(X_test)

# Afficher les meilleurs paramètres
print("Meilleurs paramètres : ", rf_search.best_params_)

# Créer un DataFrame pour les résultats
result = pd.concat([X_test_passenger_id, pd.DataFrame(y_test_pred)], axis=1)
result.columns = ["PassengerId", "Survived"]
result["PassengerId"] = result["PassengerId"].astype(int)
result.to_csv("result.csv", index=False)







['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Meilleurs paramètres :  {'n_estimators': 300, 'min_weight_fraction_leaf': 0.0, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_leaf_nodes': 10, 'max_features': 'log2', 'max_depth': 80, 'criterion': 'entropy', 'class_weight': 'balanced', 'bootstrap': False}
